In [ ]:
!pip install pywapor --quiet

### Composites

In this notebook we'll take a closer look at how `pre_et_look` works internally. To do so, we'll be using a couple of functions that you'd normally not need to touch, because they are called automatically when you run `pywapor.pre_et_look.main`.

In [ ]:
import pywapor

folder = r"/Users/hmcoerver/Local/3_compositing"
latlim = [28.9, 29.7]
lonlim = [30.2, 31.2]
timelim = ["2021-06-01", "2021-08-01"]

Previously, we had a closer look at the keyword-argument `sources`. Here we'll look at another keyword-argument that controls how the "time_bins" or "composites" are configured. We can look at the documentation again like this.

In [ ]:
help(pywapor.pre_et_look.main)

> **Question**
>
> * Which keyword-argument control the composite length?
>
> * What is its default value?

One of the first things happening when you run `pre_et_look`, is the creation of a more precise definition of the composites (from here on also referred to as time_bins) for which `pre_et_look` will prepare the data to be used by `et_look`.

By combining the period we've selected (i.e. `timelim`) and the `bin_length` we've chosen, `pre_et_look` will create the bins using the function `pywapor.general.compositer.time_bins`. We can run it like this.

In [ ]:
bin_length = "DEKAD"

bins = pywapor.general.compositer.time_bins(timelim, bin_length)

> **Question**
>
> * What does the variable `bins` contain?
>
> * What is the length of `bins`?

In [ ]:
# Check out bins here.

After defining the bins, `pre_et_look` will start collecting the different datasources. It does this with a function called `pywapor.collect.downloader.collect_sources`. 

In a previous exercise, we saw that we can get more "information" about a level by calling the following function.

In [ ]:
lvl1_info = pywapor.general.levels.pre_et_look_levels("level_1")

`collect_sources` uses the information stored in `lvl1_info` to determine what to download and how to pre-process that data. In this exercise we don't want all the data needed by `et_look` to download. Instead, we'll only download NDVI data. To do that we'll have to remove some information from `lvl1_info`.

> **Question**
>
> * Which keys does lvl1_info contain?
>
> * Note: check the Python intro notebook if you forgot how to list a dictionaries keys.

In [ ]:
# List the keys of lvl1_info

Now we will remove all the keys, except `"ndvi"` (and we override one key-value pair).

In [ ]:
selected_sources = {k: v for k, v in lvl1_info.items() if k in ["ndvi"]}

selected_sources["ndvi"]["temporal_interp"] = False

selected_sources

> **Question**
>
> How many keys are left in `selected_sources`?

Now that we have filtered the sources, we can run the function `collect_sources` to download NDVI data.

In [ ]:
dss, selected_sources = pywapor.collect.downloader.collect_sources(folder, selected_sources, latlim, lonlim, timelim)

> **Question**
>
> What datatype is `dss` and what does it contain?

In [ ]:
# Inspect the variables dss here.

Next we'll define a new function that can plot timeseries from netcdf files.

In [ ]:
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np

def plot_timeseries(dss, latlon, composites = None, var = "ndvi", unit = "-"):
    fig = plt.figure(figsize = (7, 3), dpi=100)
    ax = fig.gca()

    if not isinstance(composites, type(None)):
        x = composites.time_bins.values
        y = composites.sel(y = latlon[0], x = latlon[1], method = "nearest")[var].values
        width = np.diff(bins) 
        ax.bar(x, y, width = width, align = "edge", color = "lightblue", edgecolor = "darkblue", zorder = 8, alpha = 0.8)

    for product, ds_fh in dss.items():
        ds = xr.open_dataset(ds_fh, decode_coords = "all")
        y = ds.sel(y = latlon[0], x = latlon[1], method = "nearest")[var].values
        x = ds.time.values
        ax.scatter(x, y, label = ".".join(product), zorder = 10)

    ax.legend()
    ax.grid(zorder = 0)
    ax.set_facecolor("lightgray")
    ax.set_xlabel("time")
    ax.set_ylabel(f"{var} [{unit}]")

> **Question**
>
> Which arguments does this function require?
>
> Which keyword-arguments can this function take?

With this function we can plot timeseries at a chosen point of interest. We can define a point by making a list with a latitude and a longitude value.

In [ ]:
latlon = [29.32301, 30.77599]
plot_timeseries(dss, latlon)

Next, we'll run the functions that turns these seperate measurements into composites (again, normally this is done internally when you run `pywapor.pre_et_look.main`). Note that we pass an empty list here for the `enhancers` argument, we'll get back to enhancers at a later point. 

In [ ]:
enhancers = []
ds_composites = pywapor.general.compositer.main(dss, selected_sources, folder, enhancers, bins)

> **Question**
> 
> * What is the datatype of `ds_composites`?
>
> * Which dimensions and sizes does `ndvi` in `ds_composites` have. Does that correspond to what you saw in the graph above.

In [ ]:
# Inspect ds_composites here

Now we can pass this dataset to the function we've created earlier as a keyword-argument. 

> **Question**
>
> Re-run the function used defined earlier, but now with the keyword-argument `composites` set to `ds_composites`.

In [ ]:
# Rerun the plotting function here.

In the graph we have just created, you should see a couple of things. First of all, there are blue-dots and orange-dots that show available NDVI measurements at our POI, coming from MOD13 and MYD13 respectively. Then there are blue bars that show the composite pixel values at this location. 

> **Question**
> 
> How many days wide are the blue bars

Each composite value is the mean of the available measurements within the respective `time_bin`. For example, most of the bars match with some of the measurements. While the first bar is exactly in between two measurements, i.e. it’s the mean of the two available measurements.

> **Question**
>
> Now lets adjust the width of the composites, to be 6 days instead of dekadal. Can you recalculate the variable `bins` with a `bin_length` of 6 days

In [ ]:
# Recalculate the bins here

> **Question**
> 
> * What is the length of `bins`?
>
> * Can you recalculate the composites?

In [ ]:
# Recalculate `ds_composites` here.

> **Question**
>
> Now make a new plot.

In [ ]:
#Make another plot here

> **Question** 
>
> Do you see anything unusual in this plot?

We can solve this by turning on temporal interpolation for NDVI. The temporal interpolation is defined inside the variable `selected_sources`.

> **Question**
>
> * Can you set the key `temporal_interp` inside `selected_sources` to `"linear"`?
>
> * Hint: near the beginnning of this exercise we have already set it to `False`.

In [ ]:
# Adjust the settings for tempoeral_interpolation of ndvi here.

Now we can run the compositer again and create another plot.

In [ ]:
ds_composites = pywapor.general.compositer.main(dss, selected_sources, folder, enhancers, bins)
plot_timeseries(dss, latlon, composites = ds_composites)

The two composites that were previously missing are now there! Note that although here we are looking at one pixel, these interpolations are done for the entire domain.

Possible values for `"temporal_interp"` are `False`, `"linear"`, `"nearest"`, `"zero"`, `"slinear"`, `"quadratic"` or `"cubic"`. But be aware that some of these can be computationally heavy.

#### Your turn!

> **Question**
> 
> * What happens when you create very small or large bins?
>
> * Try out the other interpolation methods.
>
> * What happens when you change the `"composite_type"` from `"mean"` to `"max"` or `"min"`?